# MARIAN MT

Celia Botella López

## 0. PREREQUISITOS

In [ ]:
## Si se está utilizando como entorno de trabajo Google Colab, hay que ejecutar esta celda
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/LSEGloss2SpanishText/Gloss2Text Translator

In [ ]:
!pip install sacrebleu
!pip install rouge_score
!pip install datasets
!pip install sentencepiece
!pip install accelerate
!pip install transformers
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=be0012918a7f7ae0bef624ab9f9eddcc17ac85a8f002db8b409ea1f04523b66a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1. PUESTA INICIAL

### 1.1. CARGA DEL MODELO

Se carga uno de los modelos MarianMT, concretamente el modelo preentrenado "Helsinki-NLP/opus-mt-en-es".

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_marian = "Helsinki-NLP/opus-mt-en-es"

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_marian)

model = model_init()

### 1.2. PREPROCESAMIENTO DE DATOS

Se carga el dataset que contiene pares de frases en lenguaje natural del castellano (LOE) y en lengua de signos española representada con glosas (LSE).

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
dataset_df = pd.read_csv('../Corpus/corpus-spanish-gloss-10K.csv', names=["LOE", "LSE"], encoding='utf-8')
dataset_df = dataset_df.dropna()
print("Total rows: ", dataset_df.shape[0])
dataset_df[:10]

Total rows:  10000


,LOE,LSE
0,efectuaba un vuelo entre bombay y nueva york,YORK NUEVO EFECTUAR VUELO BOMBAY
1,entre otros sitios de comercio,SITIO-PL OTRO COMERCIO
2,es un edificio exento de estilo neorrománico o...,EDIFICIO EXENTO ESTILO OBRA NEORROMÁNICO MANUE...
3,esto inició un feudo con ambos equipos interca...,PASADO ESTE ATAQUE-PL FÍSICO ÉL INICIAR FEUDO ...
4,se trata de una iglesiafortaleza construida ju...,ÉL TORRE ANTERIOR TRATAR IGLESIAFORTALEZA CONS...
5,llegó a el quinto lugar en las listas,PASADO LISTA-PL LLEGAR LUGAR QUINTO
6,con el permiso de un juzgado de menores torres...,PASADO PERMISO JUZGADO TORRE-PL MENOR ACTUAR C...
7,desde la niñez fue una apasionada por la danza,PASADO NIÑEZ APASIONADO DANZA
8,sin embargo es desterrado de el bosque por su ...,EMBARGO HERMANO SU PARTE DEMONIO DESTERRAR BOS...
9,ha escrito y es coautora de varios libros sobr...,CIENCIA ESCRIBIR COAUTORA LIBRO-PL VARIOS


El dataset se divide en dos subconjuntos: uno que se va a utilizar para entrenamiento, con 1000 pares de frases, y otro para pruebas, con 100 pares de frases.

In [ ]:
train_dataset = Dataset.from_dict(dataset_df[:1000])
test_dataset = Dataset.from_dict(dataset_df[1000:1100])
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

Estos datos, hay que prepararlos para que el modelo pueda procesarlos. Para ello, se hace uso de un tokenizador, que se encarga de darle el formato que el modelo espera como entrada. Por tanto, obtenemos el tokenizador correspondiente al modelo MarianMT previamente definido.

In [ ]:
from transformers import AutoTokenizer, MarianTokenizer

tokenizer= None
tokenizer = MarianTokenizer.from_pretrained(model_marian)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Seguidamente, se define la función _`preprocess_function()`_ que prepara los datos haciendo uso del tokenizador. Con los argumentos `max_lenght` y `truncation=True`conseguimos limitar la longitud de los datos para que se trunquen a la longitud máxima cuando lo supere.

In [ ]:
def preprocess_function(examples, max_input_length = 128, max_target_length = 128):
    inputs = examples["LSE"]
    targets = examples["LOE"]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Finalmente, se tokenizan los datos utilizando la función _`preprocess_function()`_, quedando listos para pasarselos al modelo.

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['LOE', 'LSE', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['LOE', 'LSE', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

El modelo también requiere un recopilador de datos (o data collator), por lo que se crea uno para el modelo que se va a utilizar e indicando el tokenizador que dará el formato correcto a los datos.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### 1.3. DEFINICIÓN DE MÉTRICA


Se define la función _`compute_metrics_bleu()`_ para calcular las métricas BLEU de las predicciones realizadas por un modelo.

In [ ]:
import datasets
import numpy as np

def compute_metrics_bleu(eval_preds):
    preds, labels = eval_preds
    bleu_metric = datasets.load_metric("sacrebleu")

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"score_bleu": result["score"], "bleu_1": result["precisions"][0],
              "bleu_2": result["precisions"][1], "bleu_3": result["precisions"][2], "bleu_4": result["precisions"][3]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v,4) for k, v in result.items()}

Se define la función _`compute_metrics_rouge()`_ para calcular las métricas ROUGE de las predicciones realizadas por un modelo.

In [ ]:
import nltk
nltk.download('punkt')
def compute_metrics_rouge(eval_pred):
    predictions, labels = eval_pred
    rouge_metric = datasets.load_metric("rouge")

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v,4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


La función _`compute_metrics()`_ invoca a las dos funciones (BLEU y ROUGE) para calcular dichas métricas e imprimir su valor por consola.

In [ ]:
def compute_metrics(eval_pred):
  metric_bleu = compute_metrics_bleu(eval_pred)
  metric_rouge = compute_metrics_rouge(eval_pred)

  print("\nMetric BLEU: ", metric_bleu)
  print("Metric ROUGE: ", metric_rouge)

  return  {'score': metric_bleu["score_bleu"]}

## 2. AJUSTE DE HIPERPARÁMETROS

El ajuste de hiperparámetros es la búsqueda de la configuración de los parámetros de un modelo que produzce el mejor rendimiento en el proceso de aprendizaje. Para ello, se realizan varios ensayos de entrenamiento y evaluación del modelo, probando a entrenarlo con distintos valores de hiperparámetros.


En primer lugar, hay que especificar los argumentos para personalizar el entrenamiento:
* output_dir (str) — Directorio de salida donde se escribirán las predicciones del modelo y los puntos de control.
* save_total_limit (int) *opcional* — Limita la cantidad total de puntos de control y elimina los más antiguos en output_dir.
* predict_with_generate (bool) *opcional, predeterminado=False* — Indica si se va a calcular métricas generativas (ROUGE, BLEU).
* eval_accumulation_steps (int) *opcional* — Número de pasos de predicción para acumular los tensores de salida, antes de mover los resultados a la CPU. Si no se configura, todas las predicciones se acumulan en GPU/TPU antes de trasladarse a la CPU (más rápido pero requiere más memoria).
* fp16 (bool) *opcional, predeterminado=False* — Indica si se va a usar el entrenamiento de precisión fp16 de 16 bits (mixto) en lugar del entrenamiento de 32 bits.

In [ ]:
args = Seq2SeqTrainingArguments(
    "prueba_hiperparametros-1000",
    save_total_limit=1,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=True,
)

Seguidamente, se implementa la función _`my_hp_space()`_, que devuelve una configuración aleatoria de valores para cada uno de los hiperparámetros en el rango indicado. Los hiperparámetros que se van a tratar de ajustar, junto con su rango de valores límite son los siguientes:
* learning_rate (1e-5, 1e-3). La tasa de aprendizaje permite regular los pesos respecto al gradiente de la pérdida, estableciendo la frecuencia con la que la red neuronal actualiza los valores de los pesos en el aprendizaje.
* num_train_epochs [5, 10, 15]. Número de iteraciones completas (o epochs) que se realiza sobre el conjunto de datos de entrenamiento durante el proceso de aprendizaje del modelo.
* batch_size [16, 32, 64]. El tamaño del lote indica la cantidad de muestras de datos que se utilizan cada vez que se actualizan los pesos en el entrenamiento de un modelo.
* weight_decay (1e-3, 1e-1). El decaimiento de peso es una penalización que se aplica en la actualización de los pesos, siempre que sea distinto de cero.



In [ ]:
def my_hp_space(trial):
    return{
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [5, 10, 15]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-3, 1e-1)
    }

Finalmente, se entrena el modelo invocando al método _`hyperparameter_search()`_, que realiza tantos ensayos como se especifique en el argumento `n_trials` probando con distintas combinaciones de hiperparámetros (que se eligen de forma aleatoria con la función _`my_hp_space()`_) y localizando aquella con la que se obtiene la mejor métrica.

In [ ]:
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

best_trial = trainer.hyperparameter_search(direction="maximize", n_trials=20, hp_space=my_hp_space)

Del proceso de exploración de hiperparámetros del modelo se obtiene que la combinación de valores que mejor rendimiento consigue es la siguiente:
* learning_rate = 5.97e-05
* num_epochs = 10
* batch_size = 16
* weight_decay = 0.076

## 3. ENTRENAMIENTO Y EVALUACIÓN DEL MODELO

### 3.1. Entrenamiento del modelo

Se definen los argumentos con los que se va a entrenar el modelo, utilizando los valores de hiperparámetros que mejores resultados obtuvieron en la exploración.

In [ ]:
learning_rate = 5.97e-05
num_epochs = 10
batch_size = 16
weight_decay = 0.076

args = Seq2SeqTrainingArguments(
    "experimentos-1000/full_dataset",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=1,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=True,
)

Se entrena el modelo con los argumentos predefinidos.

In [ ]:
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trained = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Score
1,No log,2.393173,1.701200
2,No log,2.142104,3.303900
3,No log,2.068105,3.803700
4,No log,2.083714,3.186100
5,No log,2.087235,3.175800
6,No log,2.113012,3.527000
7,No log,2.123296,3.397300
8,1.206100,2.142099,4.013500
9,1.206100,2.162797,3.829100
10,1.206100,2.169925,3.946500



Metric BLEU:  {'score_bleu': 1.7012, 'bleu_1': 51.3678, 'bleu_2': 18.4893, 'bleu_3': 7.6239, 'bleu_4': 3.3382, 'gen_len': 16.3}
Metric ROUGE:  {'rouge1': 54.4269, 'rouge2': 21.5611, 'rougeL': 45.362, 'rougeLsum': 45.3153, 'gen_len': 16.3}

Metric BLEU:  {'score_bleu': 3.3039, 'bleu_1': 53.0942, 'bleu_2': 22.6601, 'bleu_3': 11.3661, 'bleu_4': 6.3647, 'gen_len': 16.58}
Metric ROUGE:  {'rouge1': 59.1723, 'rouge2': 27.7898, 'rougeL': 49.88, 'rougeLsum': 49.8427, 'gen_len': 16.58}

Metric BLEU:  {'score_bleu': 3.8037, 'bleu_1': 53.7919, 'bleu_2': 24.9516, 'bleu_3': 12.848, 'bleu_4': 7.4251, 'gen_len': 16.73}
Metric ROUGE:  {'rouge1': 59.6896, 'rouge2': 29.7375, 'rougeL': 51.9938, 'rougeLsum': 51.9264, 'gen_len': 16.73}

Metric BLEU:  {'score_bleu': 3.1861, 'bleu_1': 55.9018, 'bleu_2': 25.4432, 'bleu_3': 12.8056, 'bleu_4': 7.2368, 'gen_len': 15.5}
Metric ROUGE:  {'rouge1': 58.9173, 'rouge2': 27.6608, 'rougeL': 50.9724, 'rougeLsum': 51.0202, 'gen_len': 15.5}

Metric BLEU:  {'score_bleu': 3.1

### 3.2. Evaluación del modelo

Cálculo de las métricas obtenidas de las predicciones realizadas por el modelo sobre el conjunto de datos de entrenamiento.

In [ ]:
predictions = trainer.predict(tokenized_datasets["train"])
print(predictions.metrics)


Metric BLEU:  {'score_bleu': 8.6955, 'bleu_1': 98.2326, 'bleu_2': 96.067, 'bleu_3': 94.4028, 'bleu_4': 93.0373, 'gen_len': 13.643}
Metric ROUGE:  {'rouge1': 97.8757, 'rouge2': 95.7119, 'rougeL': 97.2878, 'rougeLsum': 97.2998, 'gen_len': 13.643}
{'test_loss': 0.1156916692852974, 'test_score': 8.6955, 'test_runtime': 60.0924, 'test_samples_per_second': 16.641, 'test_steps_per_second': 1.048}


Cálculo de las métricas obtenidas de las predicciones realizadas por el modelo sobre el conjunto de datos de prueba.

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.metrics)


Metric BLEU:  {'score_bleu': 3.9465, 'bleu_1': 55.425, 'bleu_2': 26.5408, 'bleu_3': 14.128, 'bleu_4': 9.2937, 'gen_len': 15.71}
Metric ROUGE:  {'rouge1': 59.7177, 'rouge2': 30.3713, 'rougeL': 51.804, 'rougeLsum': 51.9294, 'gen_len': 15.71}
{'test_loss': 2.169924736022949, 'test_score': 3.9465, 'test_runtime': 5.4077, 'test_samples_per_second': 18.492, 'test_steps_per_second': 1.294}


### 3.3. Pruebas de traducción

Carga de datos de testeo y tokenización de los mismos.

In [ ]:
corpus_test = pd.read_csv('../Corpus/corpus-test.csv', header=0, names=["LOE", "LSE"], encoding='utf-8')
corpus_test_df = Dataset.from_dict(corpus_test)
tokenized_test = corpus_test_df.map(preprocess_function, batched=True)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Se utiliza el modelo para predecir la traducción a español de cada una de las frases cargadas en glosa.

In [ ]:
n_sent = 15
short_dataset_tokenized = Dataset.from_dict(tokenized_test[0:n_sent])
predictions = trainer.predict(short_dataset_tokenized)

for i in range(n_sent):
  print("\nInput LSE: ", corpus_test_df[i]["LSE"])
  print("Output LOE: ", tokenizer.decode(predictions.predictions[i], skip_special_tokens=True))


Metric BLEU:  {'score_bleu': 2.3702, 'bleu_1': 29.1667, 'bleu_2': 8.642, 'bleu_3': 4.5455, 'bleu_4': 3.9216, 'gen_len': 10.7333}
Metric ROUGE:  {'rouge1': 54.9251, 'rouge2': 19.9908, 'rougeL': 45.6405, 'rougeLsum': 45.873, 'gen_len': 10.7333}

Input LSE:  PASADO PEPE-NP COCHE COMPRAR PEPA-NP 
Output LOE:  compró pepa y pepe coche

Input LSE:  MAÑANA NOSOTROS LLEGAR LUGAR PREVISTO 
Output LOE:  llega a un lugar previsto en la mañana de nossostros

Input LSE:  PASADO CERVANTES-NP QUIJOTE-NP ESCRIBIR 
Output LOE:  escribió cervantes quijote

Input LSE:  JESÚS-NP CAMISA SUCIO MUCHO LLEVAR 
Output LOE:  lleva desde la casa de el sucio hasta muy lejos

Input LSE:  RUBÉN-NP FERNANDO-NP IR QUERER NO CINE 
Output LOE:  rubens no van a quedar bien los fernandos

Input LSE:  PASADO CARLOS-NP CARTA CERTIFICADO DAR ÉL 
Output LOE:  carlos le dio una carta certificada

Input LSE:  PELÍCULA APTO MENOR 
Output LOE:  la película es apto para el menor

Input LSE:  PASADO YO LESIÓN TU INFORMAR GABRIEL-N